In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import inspect
import os
import sys
import time

np.random.seed(42) # nothing special about this seed (we hope)!

In [2]:
import bnpregcluster_runjingdev.regression_mixture_lib as gmm_lib

In [3]:
reg_params = dict()
with np.load('fits/transformed_gene_regression_df7_degree3_genes10000.npz') as infile:
    reg_params['beta_mean'] = infile['transformed_beta_mean']
    reg_params['beta_info'] = infile['transformed_beta_info']
    
obs_dim = reg_params['beta_mean'].shape[1]
print(reg_params['beta_info'].shape, reg_params['beta_mean'].shape)

(7000, 9, 9) (7000, 9)


In [4]:
num_components = 30
prior_params = gmm_lib.get_base_prior_params(obs_dim, num_components)
gmm = gmm_lib.GMM(num_components, prior_params, reg_params)

In [5]:
print('Running k-means init.')
kmeans_params = \
    gmm_lib.kmeans_init(gmm.reg_params,
                        gmm.num_components, 50)
print('Done.')
init_gmm_params = dict()
init_gmm_params['centroids'] = kmeans_params['centroids']
init_gmm_params['stick_propn_mean'] = np.zeros(gmm.num_components - 1)
init_gmm_params['stick_propn_info'] = np.ones(gmm.num_components - 1)

init_x = gmm.gmm_params_pattern.flatten(init_gmm_params, free=True)

Running k-means init.
Done.


In [7]:
gmm.get_params_kl(init_gmm_params)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
gmm.conditioned_obj.reset() # Reset the logging and iteration count.
gmm.conditioned_obj.set_print_every(1)

opt_time = time.time()
gmm_opt, init_x2 = gmm.optimize(init_x, gtol=1e-2)
opt_time = time.time() - opt_time

In [25]:
# def collapse(x):
#     if type(x) is np.ndarray:
#         assert x.shape == (1,)
#         return x[0]
#     else:
#         return x
    
# collapse(np.array([1.0]))
# collapse(1.0)


1.0

In [ ]:
tic = time.time()
h_cond = gmm.update_preconditioner(init_x2)
opt_time += time.time() - tic

In [ ]:
gmm.conditioned_obj.reset()
tic = time.time()
gmm_opt, gmm_opt_x = gmm.optimize_fully(
    init_x2, verbose=True)
opt_time += time.time() - tic
print('Optimization time: {} seconds'.format(opt_time))

In [ ]:
opt_gmm_params = gmm.gmm_params_pattern.fold(gmm_opt_x, free=True)
print(opt_gmm_params.keys())
print(np.sort(opt_gmm_params['probs']))